In [79]:
from datasets import load_dataset
from transformers import AutoTokenizer
import copy

tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-125m")
tokenizer.pad_token = tokenizer.eos_token

def tokenize(batch):
    batch["input_ids"] = tokenizer(batch["query"], padding=True, truncation=True, return_tensors="pt").input_ids
    return batch

def getDataset(dataset_size=None, batch_size=32):
    
    if dataset_size is None:
        split = "train"
    else:
        split = "train[:" + str(dataset_size) + "]"
        
    ds = load_dataset("argilla/OpenHermesPreferences", split=split)
    rs = copy.deepcopy(ds)
    
    cols = list(ds[0].keys())
    
    ds = ds.rename_column("prompt", "query")
    ds = ds.remove_columns([k for k in cols if k!="query" and k!='prompt'])
    ds = ds.map(tokenize, batched=True, batch_size=batch_size)
    
    rs = rs.remove_columns([k for k in cols if k!="chosen"])
    
    return ds, rs

In [84]:
from utils.dataset import getDataset as gd

dataset_dict, response_dict = gd(10)

Map:   0%|          | 0/10 [00:00<?, ? examples/s]